In [24]:
import pandas as pd
import numpy as np

In [29]:
train_df = pd.read_csv('fraudTrain.csv', index_col=0)
test_df = pd.read_csv('fraudTest.csv', index_col=0)

### FUNCTIONS TO PREPROCESS DATA

In [26]:
def preprocess_train_test(train_df, test_df):
    def preprocess_single(df):
        df_copy = df.copy()

        # Ensure 'trans_date_trans_time' is in datetime format
        df_copy['trans_date_trans_time'] = pd.to_datetime(df_copy['trans_date_trans_time'])
        df_copy['unix_time'] = pd.to_datetime(df_copy['unix_time'], unit='s')

        # Get month of year, day of week, and hour of day for transactions
        df_copy['trans_day_of_week'] = df_copy['trans_date_trans_time'].dt.day_name()
        df_copy['trans_month'] = df_copy['trans_date_trans_time'].dt.month_name()

        # Create 'time_of_day' column
        df_copy['time_of_day'] = (df_copy['trans_date_trans_time'].dt.hour * 60 + df_copy['trans_date_trans_time'].dt.minute) / (24 * 60)

        # Calculate age in years
        df_copy['dob'] = pd.to_datetime(df_copy['dob'])
        df_copy['age'] = df_copy['trans_date_trans_time'].dt.year - df_copy['dob'].dt.year - (
            (df_copy['trans_date_trans_time'].dt.month < df_copy['dob'].dt.month) |
            ((df_copy['trans_date_trans_time'].dt.month == df_copy['dob'].dt.month) & (df_copy['trans_date_trans_time'].dt.day < df_copy['dob'].dt.day))
        )

        # Sort the DataFrame by 'cc_num' and 'trans_date_trans_time'
        df_copy.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)

        # Group by 'cc_num' and calculate the time difference
        df_copy['last_purchased'] = df_copy.groupby('cc_num')['trans_date_trans_time'].diff()

        # For the first transaction for each 'cc_num', set 'last_purchased' to NaT
        df_copy['last_purchased'] = df_copy['last_purchased'].fillna(pd.NaT)

        # Convert 'last_purchased' to numerical format (total number of seconds)
        df_copy['last_purchased_secs'] = df_copy['last_purchased'].dt.total_seconds()

        # Define the Haversine distance function
        def haversine(lat1, lon1, lat2, lon2):
            R = 6371  # Radius of the Earth in kilometers
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            return R * c

        # Calculate the distance to the last purchase
        df_copy['last_merch_lat'] = df_copy.groupby('cc_num')['merch_lat'].shift()
        df_copy['last_merch_long'] = df_copy.groupby('cc_num')['merch_long'].shift()

        df_copy['distance_last_purchase'] = haversine(df_copy['merch_lat'], df_copy['merch_long'], df_copy['last_merch_lat'], df_copy['last_merch_long'])

        # For the first transaction for each 'cc_num', set 'distance_last_purchase' to NaN
        df_copy['distance_last_purchase'] = df_copy['distance_last_purchase'].fillna(np.nan)

        # Create 'same_merchant_with_last_purchase' column
        df_copy['last_merchant'] = df_copy.groupby('cc_num')['merchant'].shift()
        df_copy['same_merchant_with_last_purchase'] = (df_copy['merchant'] == df_copy['last_merchant'])

        # Drop the temporary columns
        df_copy.drop(columns=['last_merch_lat', 'last_merch_long', 'last_merchant'], inplace=True)

        # Drop the specified columns
        df_copy.drop(columns=['last_purchased', 'city', 'street', 'last', 'first', 'job', 'cc_num', 'merchant', 'state', 'zip', 'lat', 'long', 'merch_lat', 'merch_long', 'trans_num', 'unix_time', 'dob'], inplace=True)

        # Sort the DataFrame by the original index to maintain the original order
        df_copy.sort_index(inplace=True)

        return df_copy

    train_df_preprocessed = preprocess_single(train_df)
    test_df_preprocessed = preprocess_single(test_df)

    return train_df_preprocessed, test_df_preprocessed

In [27]:
# Takes in processsed train and test data, removes all January 2019 data, and returns train, val, test
def process_fraud_data(train_df_preprocessed, test_df_preprocessed):
    # Combine the preprocessed data
    combined_df = pd.concat([train_df_preprocessed, test_df_preprocessed])

    # Remove rows where 'last_purchased_secs' is NaN
    combined_df = combined_df.dropna(subset=['last_purchased_secs'])

    # Remove all January 2019 data
    combined_df = combined_df[~((combined_df['trans_date_trans_time'].dt.year == 2019) & (combined_df['trans_date_trans_time'].dt.month == 1))]

    # Drop trans_date_trans_time
    combined_df = combined_df.drop(columns=['trans_date_trans_time'])

    # Split the combined data into train, validation, and test sets by order
    total_samples = len(combined_df)
    train_end = int(0.64 * total_samples)
    val_end = int(0.80 * total_samples)

    train_data = combined_df.iloc[:train_end]
    val_data = combined_df.iloc[train_end:val_end]
    test_data = combined_df.iloc[val_end:]

    return train_data, val_data, test_data

### Example Usage

In [30]:
train_preprocessed, test_preprocessed = preprocess_train_test(train_df, test_df)

In [31]:
train, val, test = process_fraud_data(train_preprocessed, train_preprocessed)

In [32]:
train

,category,amt,gender,city_pop,is_fraud,trans_day_of_week,trans_month,time_of_day,age,last_purchased_secs,distance_last_purchase,same_merchant_with_last_purchase
52525,grocery_net,19.46,F,1423,0,Friday,February,0.001389,20,16027.0,155.644412,False
52526,misc_net,13.01,M,471,0,Friday,February,0.002083,51,163101.0,121.187723,False
52527,gas_transport,50.02,M,471,0,Friday,February,0.003472,77,55381.0,28.230127,False
52528,entertainment,6.11,M,192805,0,Friday,February,0.004167,55,91351.0,132.898691,False
52529,grocery_net,32.14,M,18408,0,Friday,February,0.005556,46,971.0,81.694148,False
...,...,...,...,...,...,...,...,...,...,...,...,...
400885,shopping_pos,34.71,M,464,0,Monday,July,0.296528,60,3607.0,94.737479,False
400886,gas_transport,38.66,F,2376,0,Monday,July,0.297917,46,29283.0,87.403605,False
400887,misc_pos,2.84,F,52126,0,Monday,July,0.297917,89,42816.0,142.355828,False
400888,grocery_pos,120.05,M,1628,0,Monday,July,0.298611,51,42883.0,9.966721,False


In [33]:
val

,category,amt,gender,city_pop,is_fraud,trans_day_of_week,trans_month,time_of_day,age,last_purchased_secs,distance_last_purchase,same_merchant_with_last_purchase
400890,gas_transport,54.99,F,836,0,Monday,July,0.298611,37,4012.0,138.996144,False
400891,gas_transport,56.25,F,73,0,Monday,July,0.298611,84,21891.0,50.205417,False
400892,misc_pos,39.94,F,22305,0,Monday,July,0.299306,80,12629.0,65.398824,False
400893,grocery_net,53.38,F,2906700,0,Monday,July,0.299306,34,2601.0,12.922509,False
400894,shopping_net,3.54,M,23045,0,Monday,July,0.300000,60,11273.0,102.414743,False
...,...,...,...,...,...,...,...,...,...,...,...,...
799008,home,86.30,M,24840,0,Tuesday,December,0.945833,59,45582.0,112.619144,False
799009,health_fitness,61.60,F,3495,0,Tuesday,December,0.945833,31,5340.0,98.188512,False
799010,personal_care,63.14,M,76,0,Tuesday,December,0.945833,64,34631.0,166.483648,False
799011,misc_pos,32.33,F,899,0,Tuesday,December,0.946528,24,27131.0,97.198956,False


In [34]:
test

,category,amt,gender,city_pop,is_fraud,trans_day_of_week,trans_month,time_of_day,age,last_purchased_secs,distance_last_purchase,same_merchant_with_last_purchase
799013,kids_pets,50.56,M,737,0,Tuesday,December,0.947222,32,20638.0,94.540838,False
799014,kids_pets,11.09,F,139,0,Tuesday,December,0.947917,65,18859.0,82.891431,False
799015,kids_pets,41.20,M,1120,0,Tuesday,December,0.948611,62,159665.0,32.075905,False
799016,personal_care,8.65,M,13602,0,Tuesday,December,0.948611,35,8883.0,120.786297,False
799017,personal_care,59.24,F,18128,0,Tuesday,December,0.949306,29,4263.0,88.726573,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,entertainment,15.56,M,258,0,Sunday,June,0.508333,58,16781.0,112.032046,False
1296671,food_dining,51.70,M,100,0,Sunday,June,0.508333,40,7962.0,94.988187,False
1296672,food_dining,105.93,M,899,0,Sunday,June,0.508333,52,29074.0,177.104191,False
1296673,food_dining,74.90,M,1126,0,Sunday,June,0.509028,39,91018.0,118.969517,False
